# Operaciones CRUD en Riak con Python

## Conexión a Riak

In [ ]:
import riak

In [ ]:
cliente = riak.RiakClient()
print("¿Está Riak disponible?", cliente.ping())

In [ ]:
!curl -s http://localhost:8098/stats  | jq .riak_kv_version

## Insertar valores

In [ ]:
bucket = cliente.bucket("bucket_name")

In [ ]:
key1 = "key_one"
val1 = 1
key = bucket.new(key1, data = val1)
key.store()

In [ ]:
key2 = "key_two"
val2 = "Dos"
key = bucket.new(key2, data = val2)
key.store()

In [ ]:
key3 = "key_three"
val3 = {
    "val1" : 1,
    "val2" : "Dos",
    "val3" : [1, 2, 3, 4, 5],
    
}

In [ ]:
key = bucket.new(key3, data = val3)
key.store()

## Insertando imágenes

In [ ]:
image_bucket = cliente.bucket('images')

photo = image_bucket.new_from_file('image_key', './images/demo.image.jpg')
photo.store()

## Recuperando información

In [ ]:
from pprintpp import pprint as pp
bucket = cliente.bucket("bucket_name")

print("key_one:", bucket.get("key_one").data)
print("key_two:", bucket.get("key_two").data)
data3 = bucket.get("key_three").data

print("key_three: ")
pp(data3)

In [ ]:
!curl http://localhost:8098/riak/bucket_name/key_one

In [ ]:
!curl http://localhost:8098/riak/bucket_name/key_two

In [ ]:
!curl -s http://localhost:8098/riak/bucket_name/key_three | jq

http://localhost:8098/riak/images/image_key

# Tablas

In [ ]:
!pip install Faker

## Escritura en Riak

Vamos a crear un dataset de datos ficticio y lo vamos a cargar un dataframe de Pandas

In [ ]:
from faker import Faker
import pandas as pd
import json

faker = Faker()
Faker.seed(4321)
profiles = []
for i in range(10):
    profile = faker.simple_profile()
    profile['birthdate'] = profile['birthdate'].isoformat()
    profiles.append(profile) 

dataframe = pd.read_json(json.dumps(profiles, default=str))

In [ ]:
dataframe

Cada una de las filas del dataset es JSON (diccionario en Python)

In [ ]:
profiles[0]

In [ ]:
print("username:", profiles[0]['username'])
print("ser:", profiles[0]['sex'])
print("mail:", profiles[0]['mail'])

Cargamos la información en Riak en un backet llamado 'profile'-

Adicionalmente creamos 2 indices, uno por sexo y otro por email.

Los índices en Riak permiten buscar clavés de una manerá mucho más rápida que si recorrieramos todas las claves de un bucket una tras otra de forma secuencial

In [ ]:
bucket = cliente.bucket("profile")

for profile in profiles:
    riak_key = bucket.new(profile['username'], data = profile)
    riak_key.add_index('idx_mail_bin', profile['mail'])
    riak_key.add_index('idx_sex_bin', profile['sex'])
    riak_key.store()

## Lectura


Obtenemos todas los valores de la tabla y cargamos la información en Pandas

In [ ]:
rows = []
for keys in bucket.stream_keys():
    for key in keys:
        rows.append(bucket.get(key).data)

df = pd.read_json(json.dumps(rows))
df

Ahora vamos a localizar las claves del sexo "M" a través del ínidice

In [ ]:
rows = []
keys = bucket.get_index("idx_sex_bin", 'M')
for key in keys.results:
    row = bucket.get(key).data
    rows.append(row)
       
df = pd.read_json(json.dumps(rows))
df

In [ ]:
print(rows[0]) 

Ahora vamos a localizar las claves del mail "joshua01@hotmail.com" a través del ínidice

In [ ]:
rows = []
keys = bucket.get_index("idx_mail_bin", 'joshua01@hotmail.com')
for key in keys.results:
    row = bucket.get(key).data
    rows.append(row)
       
df = pd.read_json(json.dumps(rows))
df

## Listado de Buckets

In [ ]:
for bucket in cliente.get_buckets():
        print('Bucket %s' % bucket )

## Listado de valores en un Bucket

In [ ]:
for keys in bucket.stream_keys():
    print("Keys: ", keys)
    for key in keys:
        value = bucket.get(key).data
        print('Key %s: %s' % (key, value))

In [ ]:
for key in bucket.get_keys():
    value = bucket.get(key).data
    print('Key %s: %s' % (key, value))